# Poker

## The Basics

Source: https://www.contrib.andrew.cmu.edu/~gc00/reviews/pokerrules

- Poker is played from a standard deck of 52 cards
- Individual cards are ranked, from high to low, as Ace, King, Queen, Jack, 10, 9, 8, 7, 6, 5, 4, 3, 2, \[Ace\] (Ace can be high or low, but is usually high).
- There are four suits: Spades, Hearts, Diamonds, and Clubs. No one suit is higher than another.
- All poker hands contain five cards, the highest hand wins.

## Hand Ranks

Hands are ranked from hight to low as follows:

1. Straight Flush
    * A Straight Flush is the best natural hand. A straight flush is a straight (5 cards in order, such as 5-6-7-8-9) that are all of the same suit. As in a regular straight, you can have an ace either high (A-K-Q-J-T) or low (5-4-3-2-1). However, a straight may not 'wraparound'. (Such as K-A-2-3-4, which is not a straight). An Ace high straight-flush is called a Royal Flush and is the highest natural hand.

2. Four of a Kind
    * Four of a Kind is simply four cards of the same rank. If there are two or more hands that qualify, the hand with the higher-rank four of a kind wins. Ties are not possible for Four of a Kind with a standard deck. However, as a general rule: when hands tie on the rank of a pair, three of a kind, etc, the cards outside break ties following the High Card rules.
    
3. Full House

    * A full house is a three of a kind and a pair, such as K-K-K-5-5. Ties are broken first by the three of a kind, then pair. So K-K-K-2-2 beats Q-Q-Q-A-A, and while not possible with a standard deck, Q-Q-Q-A-A would beat Q-Q-Q-J-J from a ranking perspective.

4. Flush

    * A Flush is a hand where all of the cards are the same suit, such as J-8-5-3-2, all of Spades. When flushes tie, follow the rules for High Card.

5. Straight

    * A Straight is 5 cards in order, such as 4-5-6-7-8. An ace may either be high (A-K-Q-J-T) or low (5-4-3-2-1). However, a straight may not 'wraparound'. (Such as Q-K-A-2-3, which is not a straight). When straights tie, the highest straight wins. (AKQJT beats KQJT9 down to 5432A). If two straights have the same value (AKQJT vs AKQJT) they split the pot.

6. Three of a Kind

    * Three cards of any rank, matched with two cards that are not a pair, otherwise it would be a Full House. Highest three of a kind wins.

7. Two Pair

    * This is two distinct pairs of card and a 5th card. The highest pair wins ties. If both hands have the same high pair, the second pair wins. If both hands have the same pairs, the high card wins.

8. Pair

    * One pair with three distinct cards. High card breaks ties.

9. High Card

    * This is any hand which doesn't qualify as any one of the above hands. If nobody has a pair or better, then the highest card wins. If multiple people tie for the highest card, they look at the second highest, then the third highest etc. High card is also used to break ties when the high hands both have the same type of hand (pair, flush, straight, etc).

In [118]:
import pandas as pd
import numpy as np
import itertools as it
from math import comb
from random import sample, seed
from collections import Counter

In [48]:
total_hands = comb(52,5)

rows = [['straight flush', 10, 40],
        ['four of a kind', 156, 624],
        ['full house', 156, 3744],
        ['flush', 1277, 5108],
        ['straight', 10, 10200],
        ['three of a kind', 858, 54912],
        ['two pair', 858, 123552],
        ['one pair', 2860, 1098240],
        ['high card', 1277, 1302540]]

hand_prob_df = pd.DataFrame(rows, columns=['hand', 'distinct_hands', 'frequency'])

hand_prob_df['probability'] = hand_prob_df.frequency / total_hands
hand_prob_df['cum_prob'] = hand_prob_df.probability.cumsum()
hand_prob_df['odds_against'] = (1.0 - hand_prob_df.probability) / hand_prob_df.probability

hand_prob_df

,hand,distinct_hands,frequency,probability,cum_prob,odds_against
0,straight flush,10,40,0.000015,0.000015,64973.000000
1,four of a kind,156,624,0.000240,0.000255,4164.000000
2,full house,156,3744,0.001441,0.001696,693.166667
3,flush,1277,5108,0.001965,0.003661,507.801879
4,straight,10,10200,0.003925,0.007586,253.800000
5,three of a kind,858,54912,0.021128,0.028715,46.329545
6,two pair,858,123552,0.047539,0.076254,20.035354
7,one pair,2860,1098240,0.422569,0.498823,1.366477
8,high card,1277,1302540,0.501177,1.000000,0.995301


In [23]:
def build_deck():
    ranks = list(range(2,15))
    suits = ['H','S','C','D']
    deck = []
    for i in ranks:
        for s in suits:
            card = s+':'+str(i)
            deck.append(card)
    return deck

In [137]:
def is_flush(hand):
    suits = [c.split(':')[0] for c in hand]
    return True if len(set(suits)) == 1 else False

def are_ranks_straight(ranks):
    straight = True
    for i in range(1,5):
        if ranks[i] - ranks[i-1] != 1:
            straight = False
            break
        else:
            pass
    return straight

def is_straight(hand):
    ranks = sorted([int(c.split(':')[1]) for c in hand])

    straight = are_ranks_straight(ranks)
    
    if 14 in ranks:
        ranks = sorted([1 if c == 14 else c for c in ranks])
        straight = are_ranks_straight(ranks)

    return straight

def is_straight_flush(hand):
    return True if is_straight(hand) & is_flush(hand) else False

def is_4oaK(hand):
    ranks = [int(c.split(':')[1]) for c in hand]
    return True if 4 in Counter(ranks).values() else False

def is_full_house(hand):
    ranks = [int(c.split(':')[1]) for c in hand]
    vals =  Counter(ranks).values()
    return True if (3 in vals) & (2 in vals) else False

def is_3oaK(hand):
    ranks = [int(c.split(':')[1]) for c in hand]
    return True if 3 in Counter(ranks).values() else False

def is_2pair(hand):
    ranks = [int(c.split(':')[1]) for c in hand]
    return True if 2 == Counter(Counter(ranks).values())[2] else False

def is_1pair(hand):
    ranks = [int(c.split(':')[1]) for c in hand]
    return True if 1 == Counter(Counter(ranks).values())[2] else False

def compare_straight_flush(hand1, hand2):
    ranks1 = sorted([int(c.split(':')[1]) for c in hand1])
    ranks2 = sorted([int(c.split(':')[1]) for c in hand2])
    # Unfinished

def compare_4oaK(hand1, hand2):

In [145]:
deck = build_deck()
draws = 10000000
seed(1234)

counts = {
    'straight flush' : 0,
    'four of a kind' : 0,
    'full house' : 0,
    'flush' : 0,
    'straight' : 0,
    'three of a kind' : 0,
    'two pair' : 0,
    'one pair' : 0,
    'high card' : 0
}

for i in range(draws):
    hand = sample(deck, 5)

    if i % 1000 == 0:
        print("{}%".format(100*i/draws))
    
    if is_straight_flush(hand):
        counts['straight flush'] += 1
        continue
    elif is_4oaK(hand):
        counts['four of a kind'] += 1
        continue
    elif is_full_house(hand):
        counts['full house'] += 1
        continue
    elif is_flush(hand):
        counts['flush'] += 1
        continue
    elif is_straight(hand):
        counts['straight'] += 1
        continue
    elif is_3oaK(hand):
        counts['three of a kind'] += 1
        continue
    elif is_2pair(hand):
        counts['two pair'] += 1
        continue
    elif is_1pair(hand):
        counts['one pair'] += 1
        continue
    else:
        counts['high card'] += 1

0.7465%
0.7466%
0.7467%
0.7468%
0.7469%
0.747%
0.7471%
0.7472%
0.7473%
0.7474%
0.7475%
0.7476%
0.7477%
0.7478%
0.7479%
0.748%
0.7481%
0.7482%
0.7483%
0.7484%
0.7485%
0.7486%
0.7487%
0.7488%
0.7489%
0.749%
0.7491%
0.7492%
0.7493%
0.7494%
0.7495%
0.7496%
0.7497%
0.7498%
0.7499%
0.75%
0.7501%
0.7502%
0.7503%
0.7504%
0.7505%
0.7506%
0.7507%
0.7508%
0.7509%
0.751%
0.7511%
0.7512%
0.7513%
0.7514%
0.7515%
0.7516%
0.7517%
0.7518%
0.7519%
0.752%
0.7521%
0.7522%
0.7523%
0.7524%
0.7525%
0.7526%
0.7527%
0.7528%
0.7529%
0.753%
0.7531%
0.7532%
0.7533%
0.7534%
0.7535%
0.7536%
0.7537%
0.7538%
0.7539%
0.754%
0.7541%
0.7542%
0.7543%
0.7544%
0.7545%
0.7546%
0.7547%
0.7548%
0.7549%
0.755%
0.7551%
0.7552%
0.7553%
0.7554%
0.7555%
0.7556%
0.7557%
0.7558%
0.7559%
0.756%
0.7561%
0.7562%
0.7563%
0.7564%
0.7565%
0.7566%
0.7567%
0.7568%
0.7569%
0.757%
0.7571%
0.7572%
0.7573%
0.7574%
0.7575%
0.7576%
0.7577%
0.7578%
0.7579%
0.758%
0.7581%
0.7582%
0.7583%
0.7584%
0.7585%
0.7586%
0.7587%
0.7588%
0.7589%
0.759%
0.7591

In [146]:
[v/draws for v in counts.values()]

[1.53e-05,
 0.0002468,
 0.0014412,
 0.0019526,
 0.0035297,
 0.0210717,
 0.0475147,
 0.4224606,
 0.5017674]

In [ ]:
def classify_hand(hand):
    if is_straight_flush(hand):
        return 1
    elif is_4oaK(hand):
        return 2
    elif is_full_house(hand):
        return 3
    elif is_flush(hand):
        return 4
    elif is_straight(hand):
        return 5
    elif is_3oaK(hand):
        return 6
    elif is_2pair(hand):
        return 7
    elif is_1pair(hand):
        return 8
    else:
        return 9

def compare_hands(hand1, hand2):
    # return 1 if hand1 <= hand2 else return 0
    hand1, hand2 = list(hand1), list(hand2)
    if classify_hand(hand1) < classify_hand(hand2):
        return 1
    elif classify_hand(hand1) > classify_hand(hand2):
        return 0
    else:
        if is_straight_flush(hand1):
            return compare_straight_flush(hand1, hand2)
        elif is_4oaK(hand1):
            pass
        elif is_full_house(hand1):
            pass
        elif is_flush(hand1):
            pass
        elif is_straight(hand1):
            pass
        elif is_3oaK(hand1):
            pass
        elif is_2pair(hand1):
            pass
        elif is_1pair(hand1):
            pass
        else


In [152]:
all_hands = list(it.combinations(deck,5))

In [155]:
def QuickSort(arr):

    elements = len(arr)
    
    #Base case
    if elements < 2:
        return arr
    
    current_position = 0 #Position of the partitioning element

    for i in range(1, elements): #Partitioning loop
         if arr[i] <= arr[0]:
              current_position += 1
              temp = arr[i]
              arr[i] = arr[current_position]
              arr[current_position] = temp

    temp = arr[0]
    arr[0] = arr[current_position] 
    arr[current_position] = temp #Brings pivot to it's appropriate position
    
    left = QuickSort(arr[0:current_position]) #Sorts the elements to the left of pivot
    right = QuickSort(arr[current_position+1:elements]) #sorts the elements to the right of pivot

    arr = left + [arr[current_position]] + right #Merging everything together
    
    return arr



array_to_be_sorted = [4,2,7,3,1,6]
print("Original Array: ",array_to_be_sorted)
print("Sorted Array: ",QuickSort(array_to_be_sorted))

['H:2', 'S:2', 'C:2', 'D:2', 'H:3']

In [157]:
max([2,4,7])

7